In [1]:
#| default_exp bridge_helpers

In [2]:
#| export
from __future__ import annotations

# Bridge helpers


# Prologue

In [3]:
#| export
import os
import time
from collections import defaultdict
from pathlib import Path
from typing import Any
from typing import Callable

import anywidget
import fastcore.all as FC
import traitlets as T
from fastcore.all import L
from fastcore.all import NotStr
from fastcore.xml import escape
from fastcore.xml import to_xml
from fasthtml.core import FT
from fasthtml.jupyter import JupyUvi
from fasthtml.xtend import Script
from IPython.display import display
from IPython.display import HTML
from olio.basic import bundle_path
from olio.common import update_
from traitlets.config import SingletonConfigurable


In [4]:
os.environ['DEBUG'] = 'True'

In [5]:
#| export

from fasthtml.components import Div,B, Details, Summary, Pre

In [6]:
#| export
from bridget.display_helpers import LogCanvas
from bridget.display_helpers import LogCanvas_css
from bridget.display_helpers import NBLogger
from bridget.display_helpers import NoopLogger
from bridget.helpers import bridge_cfg
from bridget.helpers import id_gen
from bridget.widget_helpers import anysource
from bridget.widget_helpers import BlockingMixin
from bridget.widget_helpers import bundled

In [7]:
import asyncio
import inspect
import json
import operator
import sys
from contextlib import contextmanager
from inspect import Parameter 
from typing import DefaultDict
from typing import Iterable
from typing import Protocol
from typing import Self

import anyio
import fasthtml.components as ft
from anyio import from_thread
from fastcore.test import *
from fasthtml.components import highlight
from fasthtml.components import show
from fasthtml.components import showtags
from fasthtml.core import FastHTML
from fasthtml.core import fh_cfg
from fasthtml.core import fhjsscr
from fasthtml.core import scopesrc
from fasthtml.core import surrsrc
from httpx import get
from IPython.core.displayhook import CapturingDisplayHook
from IPython.core.getipython import get_ipython
from IPython.core.interactiveshell import InteractiveShell
from IPython.display import clear_output
from IPython.display import DisplayHandle
from IPython.display import Javascript
from IPython.display import Markdown
from IPython.utils.capture import capture_output
from IPython.utils.capture import CapturedIO
from IPython.utils.capture import RichOutput
from loguru import logger
from olio.common import gets
from olio.common import pops_
from olio.common import pops_values_
from olio.common import setup_console
from olio.common import shortens
from olio.test import test_raises
from olio.widget import cleanupwidgets


In [8]:
from bridget.display_helpers import BasicLogger
from bridget.display_helpers import displaydh
from bridget.display_helpers import LogCanvas_stl
from bridget.helpers import kounter
from bridget.widget_helpers import ScriptV

In [9]:
from fasthtml.components import Div, P, Pre, Text, Span

----


In [10]:
#| exporti

new_id = id_gen()
DEBUG = os.environ.get('DEBUG', None) == 'True'
BUNDLE_PATH = bundle_path(__name__)

In [11]:
IDISPLAY = display
_n = '\n'
console, cprint = setup_console(140)

In [12]:
# needed for vfile:
%load_ext anywidget

In [13]:
%env ANYWIDGET_HMR=0

env: ANYWIDGET_HMR=0


----

In [14]:
# %%HTML
# <script type="module">
# debugger;
# const mod = await import('../bridget/test.js');
# const { test } = mod;
# console.log(test);
# </script>

In [15]:
# %%HTML
# <script type="module">
# debugger;
# const { default: getObserverManager } = await import('./observer.js');
# console.log(getObserverManager);
# const observerManager = getObserverManager();
# </script>

In [16]:
# %%HTML
# <script type="module">
# debugger;
# const { getObserverManager } = await import('observer.js');
# console.log(getObserverManager);
# getObserverManager();
# </script>

# Helpers

In [17]:
#| export

def debug(jsstr: str): return f"debugger;\n{jsstr}"
def notdebug(jsstr: str): return jsstr

In [18]:
#| export

def _to_js(ft):
    if isinstance(ft, FT): 
        return [ft.tag, [_to_js(_) for _ in ft.children], ft.attrs]
    return str(ft)

def to_js(*fts: FT|str):
    return f"[\n  {',\n  '.join(repr(_to_js(_)) if isinstance(_, FT) else repr(_) for _ in fts)}\n]"

In [19]:
a = Div(P('a'), P('aa'), id='a11')
b = P('b', Span('bb'))

test_eq(_to_js(a), ['div', [['p', ['a'], {}], ['p', ['aa'], {}]], {'id': 'a11'}])
test_eq(_to_js(b), ['p', ['b', ['span', ['bb'], {}]], {}])

test_eq(to_js(a, 'const a=`${"a"}`; console.log(a)', b), """[
  ['div', [['p', ['a'], {}], ['p', ['aa'], {}]], {'id': 'a11'}],
  'const a=`${"a"}`; console.log(a)',
  ['p', ['b', ['span', ['bb'], {}]], {}]
]""")

In [20]:
a = Script('const a=`${"a"}`; console.log(a)')
b = Script(src='https://unpkg.com/htmx.org@next/dist/htmx.js', type='module')

test_eq(repr(_to_js(a)), "['script', ['const a=`${\"a\"}`; console.log(a)'], {}]")

test_eq(to_js(a, b), """[
  ['script', ['const a=`${"a"}`; console.log(a)'], {}],
  ['script', [''], {'src': 'https://unpkg.com/htmx.org@next/dist/htmx.js', 'type': 'module'}]
]""")

Quick & dirty way to convert `FT` to HTML elements int the front-end using `fasthtml-js` `$E`.  
Intended only for "linking"/head elements (with void or text content): script, style, link, meta, etc. It'll surely fail with other elements.

In [21]:
#| export

class ScriptsDetails:
    def __init__(self, scs, title='Loaded scripts', open=True): 
        self.scs = scs; self.title = title; self.open = open
    def __ft__(self):
        return Details(open=self.open)(
            Summary(B(self.title)),
            Pre(NotStr('\n'.join(escape(to_xml(_, indent=False, do_escape=False).strip()) for _ in self.scs))),
        )

# autoid (not used)

Note: `autoid` here is different from `fh_cfg['auto_id']` option. Here we're trying to automatically set the `id` attribute of the wrapper element of a cell output. If we can do so, we'll be able to target especific cell outputs down the road.

In [22]:
# %%vfile autoid.js

# // debugger;
# function autodel(id) {
#     // debugger;
#     const me = document.querySelector(`#${id}`);
#     const parent = me.parentElement;
#     // parent.append(`test ${id}`);
#     window.setTimeout(() => {{ 
#         me.remove(); 
#         parent.setAttribute("id", `output-${id}`);
#         console.log('deleted script', id); 
#     }}, 100);
#     console.log("test script", id);
# }

In [23]:
# sc = '''
# function autodel(id) {
#     // debugger;
#     const me = document.querySelector(`#${id}`);
#     const parent = me.parentElement;
#     // parent.append(`test ${id}`);
#     window.setTimeout(() => {{ 
#         me.remove(); 
#         parent.setAttribute("id", `output-${id}`);
#         console.log('deleted script', id); 
#     }}, 100);
#     console.log("test script", id);
# }
# '''

# # autodelscr = Script(sc)
# autodelscr = ScriptV('vfile:autoid.js')
# HTML(to_xml(autodelscr))

In [24]:
# sc = '''
# autodel("{0}");
# '''

# idx = new_id()
# scr = Script(notdebug(sc.format(idx)), id=idx, type='module')
# HTML(to_xml(scr)+'aaaa')

In [25]:
# sc = '''
# autodel("{0}");
# '''
# def autoid(idx=None):
#     idx = idx or new_id()
#     return Script(notdebug(sc.format(idx)), id=idx, type='module'), idx

In [26]:
# HTML('aaaaa'+to_xml(autoid()[0]))

In [27]:
# %%HTML
#   <script id="asdfg-12345">
#     debugger;
#     me().attribute('id', 'output-asdfg-12345');
#     setTimeout(el => { el.remove(); }, 100, me('#asdfg-12345'))
#     // autodel('asdfg-12345');
#   </script>
#   <div class="uploader">aaaa</div>

In [28]:
# __autoid_scr = '''
# me().attribute('id', 'output-{0}').classAdd('bridge');
# setTimeout(el => {{ el.remove(); }}, 100, me('#{0}'))
# '''
# def autoid(idx=None):
#     idx = idx or new_id()
#     return Script(notdebug(__autoid_scr.format(idx)), id=idx), idx

In [29]:
# scr,idx = autoid()
# dhdl = DisplayHandle(idx)
# dhdl.display(HTML('bbbb'+to_xml(scr)))

In [30]:
# dhdl.update(HTML('cccc'+to_xml(scr)))

In [31]:
#| export

bridge_js = BUNDLE_PATH / 'js/bridge.js'
bridge_esm = bundled(bridge_js, bundle=__name__, bundler='copy')

# Bridge bootstrap

## BridgeBoot

In [32]:
#| exporti

class BridgeBoot(anywidget.AnyWidget, BlockingMixin, SingletonConfigurable):
    _esm = anysource(bridge_esm(debugger=DEBUG), '''
export default { 
    initialize({ model }) {
        $Brd.model = model;
        return () => $Brd.model = null;
    }
}
''')
    _css = LogCanvas_css

    def __init__(self, *args, **kwargs):
        self.on_msg(self.on_init_message)
        super().__init__(*args, **kwargs)
    @classmethod
    async def create(cls, *args, **kwargs):
        return await super().create(*args, factory=cls.instance, **kwargs)
    def loaded(self): return not self._loading
    def on_init_message(self, *_):
        "One-time init msg handler"
        self._loading = False
        self.on_msg(self.on_init_message, remove=True)
        time.sleep(0.1)
        self.close()

In [33]:
cleanupwidgets('bridge', clear=False)

bridge = await BridgeBoot.create()
test_eq(bridge.comm, None)

In [34]:
test_eq(bridge.loaded(), True)

In [35]:
#| exporti

async def get_bridge(*args, **kwargs):
    if not BridgeBoot.initialized():
        bridge = await BridgeBoot.create(*args, **kwargs)
        if not bridge.loaded(): bridge = None
    else: 
        if not BridgeBoot.instance().loaded(): BridgeBoot.clear_instance()
        bridge = BridgeBoot.instance()
    return bridge

# def get_bridge(*args, **kwargs):
#     async def _arun(): return await _get_bridge(*args, **kwargs)
#     with from_thread.start_blocking_portal() as portal: return portal.call(_arun)

In [36]:
test_is(await get_bridge(), bridge)

## handle_message

In [37]:
#| export

def handle_message(
        o: Any, 
        *args: Any, 
        ctx:str, kind:str, 
        prefix:str='on_', forward:bool=True, forward_name:str='_msg_hndlr', 
        **kwargs: Any):
    """if `o` has an attr named `ctx`, look for a handler with the form `on_{kind}` 
    passing the rest of `msg` and `args` to it. 
    If `forward`and `o` has an attr named `forward_name`, call it with `o`, `msg` and `args`."""
    # print(f"handle_message: {o=} {args=} {ctx=}, {kind=} {kwargs=}")
    if ctx in getattr(o, 'ctx_names', ()) and (fn := getattr(o, f"{prefix}{kind}", None)): fn(*args, **kwargs)
    if forward and (fn := getattr(o, forward_name, None)): fn(o, *args, ctx=ctx, kind=kind, **kwargs)

In [38]:
class O:
    ctx_names=set({'test'})
    def on_info(self, *args, **kwargs): print('info', args, kwargs)

o = O()
handle_message(o, 'hello', ctx='test', kind='info', info='initialized')

info ('hello',) {'info': 'initialized'}


## BridgeWidget
> Base class of widgets that setup or augment the bridge.

In [39]:
#| export

class BridgeWidget(anywidget.AnyWidget, BlockingMixin):
    _esm = anysource('''
export default { 
    initialize({ model }) {
        $Brd.model = model;
        model.send({ ctx: model.get('ctx_name'), kind: 'info', info: 'loaded' });
        return () => $Brd.model = null;
    }
}
''')

    ctx_name = T.Unicode('bridge').tag(sync=True)
    ctx_names: set[str]
    logger = T.Instance(NBLogger, default_value=NoopLogger())
    
    def __init__(self, msg_hndlr:Callable|None=None, **kwargs):
        self._loading, self._pending, self._msg_hndlr = True, defaultdict(dict), msg_hndlr
        self.on_msg(self.on_init_message)
        super().__init__(**kwargs)
        self.logger.log(f"Loading {self.__class__.__name__}...")
        # Warning: AnyWidget calls `self.add_traits()`: self.__class__ before 
        # super()__init__ is not self.__class__ now
        self.ctx_names = set(L(type(self).mro()).attrgot('ctx_name.default_value').filter())
    
    @classmethod
    async def create(cls, *args, **kwargs):
        if not await get_bridge(): raise RuntimeError("Bridge is not loaded.")
        self = await super().create(*args, **kwargs)
        if self._loading: self.logger.log(f"Timeout loading {cls.__name__}@{hex(id(self))}")
        else: self.logger.log(f"----")
        return self

    def msg(self, tracker:Any=None, **kwargs): 
        msg_id = new_id()
        if tracker: self._pending[msg_id] = tracker
        return update_(kwargs, msg_id=msg_id)

    def on_info(self, *args, info:str, **kwargs):
        "Handle info messages from the front-end."
        if info == 'loaded':
            self.logger.log(f"{self.__class__.__name__}@{hex(id(self))} loaded.")
            self.logger.log(str(vars(bridge_cfg)))
        if info == 'model-setup':
            self.logger.log(f"{self.__class__.__name__}@{hex(id(self))} model set.")
        if info == 'model-unset':
            self.logger.log(f"{self.__class__.__name__}@{hex(id(self))} model unset.")

    def on_error(self, *args, error:str, **kwargs):
        "Handle error messages from the front-end."
        self.logger.log(f"{self.__class__.__name__}@{hex(id(self))} error: {error}")

    def on_init_message(self, _, content, buffers):
        "One-time init msg handler"
        self._loading = False
        self.on_msg(self.on_init_message, remove=True)
        self.on_msg(self.on_message)
        handle_message(self, buffers, **content)
    
    def on_message(self, _, content, buffers):
        tracker = self._pending[msg_id] if (msg_id := content.get('msg_id')) else None
        handle_message(self, buffers, **content, tracker=tracker)
        if msg_id and not tracker: self._pending.pop(msg_id)

In [40]:
cleanupwidgets('brd', clear=False)
# BridgeBoot._loading = True

brd = await BridgeWidget.create(logger=BasicLogger(height=100))

In [41]:
# brd.close()

# Linking elements loader
> `script` (and other links) loader for notebooks.

With standard web apps, `FastHTML` way of loading `head` elements is fine, `head` links are execute in order (unless they have `async` or `defer` attributes) if present in HTML source.

In notebooks, we need to load those `head` elements dinamically in order (in `head`, `body` or anywhere of the front-end page), and we need to load `fasthtml.js` (and `htmx.js` because why not) as soon as possible, so we can use them in the same cell to define our JS extensions.


In [42]:
#| export

def Links(*fts: FT):
    return Script(notdebug("""
if (window.$Ls) requestAnimationFrame(() => $Ls(%s));
""" % to_js(*fts)), type='module')

In [43]:
#| export

def load_links(*fts: FT, feedback: str=''):
    dh = display(HTML(to_xml(Links(*fts)) + feedback), display_id=True)
    # if not feedback and dh: dh.update(HTML(''))
    # if not feedback: clear_output()

In [44]:
scr = Script(debug('''
$Brd.logger.log('silly script');
'''), id='silly-script')
load_links(scr)

In [45]:
load_links(scr)  # see console, script was not loaded twice

`load_links` can be used to load any link element in the front-end in order. It will auto delete the loader after links are loaded if `feedback` is None so the link won't be reflected in the `.ipynb` file and loaded automatically on page open.

Unfortunately, in some Jupyter environments like VSCode this only works if the cell is visible in the screen and is run interactively, not for `all below` or `all above`. VSCode only renders outputs that are visible. For an alternative, see `Loader` below.


# BLogger

In [46]:
#| export

defaultLogConfig = {
    'ns': 'brd',
    'color': '#000000',
    'fmt': 'htmlFmt',
    'tsDelta': True,
    'INFO': {
        'level': 'INFO',
    },
    'ERROR': {
        'level': 'ERROR',
        'color': '#FF0000',
    },
}

logger_js = BUNDLE_PATH / 'js/logger.js'
logger_esm = bundled(logger_js, bundle=__name__, bundler='copy')

class BLogger(anywidget.AnyWidget, NBLogger):
    "Configure a LogCanvas as the Bridge logger."
    _esm = anysource(logger_esm(debugger=DEBUG), '''
export default { 
    initialize({ model }) { return initLogger(model); }
};
''')
    
    canvas: LogCanvas
    logger_config = T.Dict(default_value=defaultLogConfig).tag(sync=True)
    elid = T.Unicode('').tag(sync=True)
    
    def __init__(self, canvas:LogCanvas|None=None, show:bool=True, **kwargs):
        self.canvas = canvas or LogCanvas()
        T.link((self.canvas, 'elid'), (self, 'elid'))
        if show: 
            def _on_init_message(self, *_):
                self.on_msg(_on_init_message, remove=True)
                self.display()
            self.on_msg(_on_init_message)
        super().__init__(**kwargs)

    def display(self, content:str=''): self.canvas.display(content)
    
    # @T.observe('canvas')
    # def canvas_changed(self, change):
    #     if change['new']: 
    #         self.logger_config = { **self.logger_config, 'elid': change['new'].elid }

    def config(self, **kwargs): self.logger_config = { **self.logger_config, **kwargs }
    def log(self, msg:str, clear:bool=False):
        self.send({'rec':{'message':msg, 'level': 'INFO'}, 'clear':clear or None})
    def clear_log(self): self.send({'clear':True})
    def close(self, msg:str|None=None):
        if self.comm is not None:
            self.send({'rec':{ 'message':msg if msg else 'logger closed.', 'level': 'INFO' }})
            self.send = FC.noop  # type: ignore
            super().close()

In [47]:
cleanupwidgets('lgr')

lgr = BLogger()

In [50]:
lgr.log('bbbb')

In [51]:
lgr.log('cccccc', True)


In [52]:
lgr.display()

In [53]:
lgr.log('ddddddddd')

In [54]:
brd.logger = lgr

In [55]:
brd.logger.log('test1')

In [56]:
Javascript('''$Brd.logger.log('test2');''')

<IPython.core.display.Javascript object>

In [57]:
lgr.close()

In [58]:
canvas = LogCanvas(height=200)
canvas.display(f"{'<br>'.join('def')}<br>")

In [59]:
cleanupwidgets('lgr')

lgr = BLogger(canvas)

In [60]:
lgr.log('gggg')

In [61]:
lgr.clear_log()

In [62]:
# lgr.close()

# Loader
> AnyWidget loader for JS modules.

Load also bridge-js to bootstrap the bridge.


In [63]:
#| export

loader_js = BUNDLE_PATH / 'js/loader.js'
loader_esm = bundled(loader_js, bundle=__name__, bundler='copy')

class Loader(BridgeWidget, SingletonConfigurable):
    _esm = anysource(loader_esm(debugger=DEBUG), '''
export default { 
    initialize({ model }) { 
        $Brd.model = model;
        initializeLoader(model);
        model.send({ ctx: model.get('ctx_name'), kind: 'info', info: 'loaded' });
        return () => $Brd.model = null;
    }
};
''')

    ctx_name = T.Unicode('loader').tag(sync=True)
    lnks = {}
    _loaded = T.Set()

    def __init__(self, 
            lnks: dict[str, FT]|None = None, 
            esms: list[str|Path]|None = None, 
            *, logger:NBLogger|None=None, **kwargs
        ):
        if lnks: self.lnks = lnks
        if logger is None: logger = BLogger(canvas=LogCanvas())
        logger.display()
        super().__init__(logger=logger, **kwargs)

    @classmethod
    async def create(cls, *args, **kwargs):
        return await super().create(*args, factory=cls.instance, **kwargs)

    def on_info(self, *args, info:str, **kwargs):
        super().on_info(*args, info=info, **kwargs)
        if info == 'loaded': # now we can initialize the scripts
            self.load_links(**self.lnks)

    def loaded(self, name:str|None=None):
        if not name: return not self._loading
        return name in self._loaded
    
    def on_load(self, *args, success:list[str], failed:list[dict[str,str]], tracker:dict, **kwargs):
        for name in success: self.logger.log(f"'{name}' loaded.")
        for res in failed: self.logger.log(f"'{res['name']}' failed: {res['error']}")
        self._loaded = self._loaded.union(success)
        
    def on_loadLinks(self, *args, success:list[str], failed:list[dict[str,str]], tracker:dict, **kwargs):
        for name in success: self.logger.log(f"'{name}' loaded.")
        for res in failed: self.logger.log(f"'{res['name']}' failed: {res['error']}")
        self._loaded = self._loaded.union(success)

    def load(self, **esms: str|Path):
        if not esms: return
        msg = self.msg(esms, ctx='loader', cmd='load', 
            args={n:esm.read_text() if isinstance(esm, Path) else esm for n,esm in esms.items()})
        self.send(msg)
    def load_links(self, **lnks: FT):
        if not lnks: return
        msg = self.msg(lnks, ctx='loader', cmd='loadLinks', args={n:_to_js(_) for n,_ in lnks.items()})
        self.send(msg)

    async def _asend(self, msg, kind:str):
        res = await self.asend(msg)
        try:
            msg_id = res[0]['msg_id']  # type: ignore
            if msg_id in self._pending: self.logger.log(f"Timeout loading {kind} {msg_id}")
        except Exception: pass
    async def aload(self, **esms: str|Path):
        if not esms: return
        msg = self.msg(esms, ctx='loader', cmd='load', 
            args={n:esm.read_text() if isinstance(esm, Path) else esm for n,esm in esms.items()})
        await self._asend(msg, 'ESMs')
    async def aload_links(self, **lnks: FT):
        if not lnks: return
        msg = self.msg(lnks, ctx='loader', cmd='loadLinks', args={n:_to_js(_) for n,_ in lnks.items()})
        await self._asend(msg, 'links')

In [64]:
cleanupwidgets('loader')
Loader.clear_instance()

ldr = await Loader.create(dict(
    test=Script('debugger;\nconsole.log("test")', id='test-script'),
    test2=Script('debugger;\nconsole.log(a)', id='test-script2')
), logger=lgr)
# ldr = Loader.instance(logger=lgr)

In [65]:
# ldr.load_links(
#     test=Script('debugger;\nconsole.log("test")', id='test-script'),
#     test2=Script('debugger;\nconsole.log(a)', id='test-script2')
# )

In [66]:
await ldr.aload_links(
    test3=Script('debugger;\nconsole.log("test")', id='test-script3'),
    test4=Script('debugger;\nconsole.log(a)', id='test-script4')
)

In [67]:
ldr.load(htmx1='''
debugger;
import htmx from "https://unpkg.com/htmx.org@next/dist/htmx.esm.js";
console.log(htmx);
''',
    htmx2='''
debugger;
import htmx from "https://unpkg.com/htmx.org@next/dist/XXXX.esm.js";
console.log(htmx);
''')

In [68]:
#| export

# def get_loader():
#     async def _arun(): return await Loader.create()
#     with from_thread.start_blocking_portal() as portal: return portal.call(_arun)
async def get_loader(*args, **kwargs): 
    if not Loader.initialized():
        return await Loader.create(*args, **kwargs)
    loader = Loader.instance()
    loader.logger.display()
    return loader

In [69]:
loader = await get_loader()

# ObserverManager
> `MutationObserver` manager for `bridget`


In [70]:
#| export

observer_js = BUNDLE_PATH / 'js/observer.js'
observer_scr = Script(debug(observer_js.read_text()), type='module', id='brd-observer-manager')

In [71]:
# observer_scr = Script(debug(f'''
# const {{ default:getObserverManager }} = await import('./{observer_js}');
# window.$Brd.observerManager = getObserverManager();
# '''), type='module')

In [72]:
# brd.load(observer_js)

In [73]:
load_links(observer_scr)

In [74]:
observer_esm = bundled(observer_js, bundle=__name__)
loader.load(observer=observer_esm(debugger=True))

# FastHTML Jupyter display

Opt-in to display FastHTML objects in the notebook as HTML instead of the default markdown `repr`.

:::{.callout-warning    }
Note recent FastHTML [v0.9.0](https://github.com/AnswerDotAI/fasthtml/releases/tag/0.9.0) has improved Jupyter support substancially and now have this same functionality (with `render_ft`, in v0.9.1 on by default).

Unfortunately, current implementation doesn't work in VSCode/Cursor and probably any other Jupyter-ish environment that sandbox notebook outputs.

In VSCode/Cursor besides that, `render_ft` doesn't work as expected, as Quarto renders markdown in a static shadow-root.
:::


## Patching FastHTML jupyter to work in VSCode

In [75]:
#| exporti

def htmx_config_port(host='localhost', port=8000):
    display(HTML('''
<script>
document.body.addEventListener('htmx:configRequest', (event) => {
    if(event.detail.path.includes('://')) return;
    htmx.config.selfRequestsOnly=false;
    event.detail.path = `http://%s:%s${event.detail.path}`;
});
</script>''' % (host, port)))

In [76]:
#| export

class JupyUviB(JupyUvi):
    "Start and stop a Jupyter compatible uvicorn server with ASGI `app` on `port` with `log_level`"
    def __init__(self, app, log_level="error", host='0.0.0.0', port=8000, start=True, **kwargs):
        self.kwargs = kwargs
        FC.store_attr(but='start')
        self.server = None
        if start: self.start()
        htmx_config_port(host, port)

In [77]:
app = FastHTML()
rt = app.route

@app.route
def index(): return 'hi'

port = 8000
server = JupyUviB(app, port=port)

In [78]:
get(f'http://localhost:{port}').text

'hi'

In [79]:
fh_cfg['auto_id']=True

In [80]:
show(Script(src='https://unpkg.com/htmx.org@2.0.4/dist/htmx.js'), fhjsscr, scopesrc, surrsrc)
clear_output()

In [81]:
display(Javascript('''
if (window.htmx) htmx.process(document.body);
'''))
clear_output()

In [82]:
#| exporti

def render_ft():
    @FC.patch
    def _repr_html_(self:FT): 
        return to_xml(
            (Div(self), 
            Script('if (window.htmx) htmx.process(document.body)'))
        )

In [83]:
render_ft()

After importing `fasthtml.jupyter` and calling `render_ft()`, FT components render directly in the notebook.

In [84]:
(c := Div('Cogito ergo sum'))

div(('Cogito ergo sum',),{'id': '_sewRlTP8SJ6bE0YlSXh0sQ'})

Handlers are written just like a regular web app:

In [85]:
@rt
def hoho(): return P('loaded!'), Div('hee hee', id=c, hx_swap_oob='true')

All the usual `hx_*` attributes can be used:

In [86]:
P('not loaded', hx_get=hoho, hx_trigger='load')

p(('not loaded',),{'hx-get': <fasthtml.core._mk_locfunc.<locals>._lf object at 0x13a783770>, 'hx-trigger': 'load', 'id': '_mozl89pQRcqxzjpJU5YBQA'})

FT components can be used directly both as `id` values and as `hx_target` values.

In [87]:
(c := Div(''))

div(('',),{'id': '_-7gkQDugT9iVn1yIqjRieg'})

In [88]:
@rt
def foo(): return Div('foo bar')

P('hi', hx_get=foo, hx_trigger='load', hx_target=c)

p(('hi',),{'hx-get': <fasthtml.core._mk_locfunc.<locals>._lf object at 0x13a2cb290>, 'hx-trigger': 'load', 'hx-target': '#_-7gkQDugT9iVn1yIqjRieg', 'id': '_mQJD2fVPSWuDFCLd36QKVw'})

## Stop server
----

In [89]:
server.stop()

----


# brd-mark

In [90]:
#| export
_all_ = ['Brd_Mark']

Brd_Mark = FT('brd-mark', (), {})

marker_js = '''
if (!customElements.get('brd-mark')) {
    class BridgetMarker extends HTMLElement {
        connectedCallback() {
            this.style.display = 'none';
            const pel = this.parentElement;
            if (pel) {
                const bridgetId = this.getAttribute('id');
                if (bridgetId) {
                    if (!pel.hasAttribute('data-brt-id')) pel.setAttribute('data-brt-id', bridgetId);
                    $Brd.logger.log('bridget marker set', bridgetId);
                    this.remove();
                }
                if (window.htmx) htmx.process(pel);
            }
        }
    }
    customElements.define('brd-mark', BridgetMarker);
    $Brd.logger.log('brd-mark defined');
} else {
    $Brd.logger.log('brd-mark is already defined');
}
'''

marker_scr = Script(notdebug(marker_js), id='brd-mark')

In [91]:
load_links(marker_scr)

In [92]:
marker = 'aaaa<brd-mark id="marker-123">'
display(HTML(marker))

In [93]:
print('asdf\nwerwqert')

asdf
werwqert


In [94]:
display(HTML('<div class="bridge">aaaa</div><brd-mark id="marker-1234"></brd-mark>'))
display(HTML('<div class="bridge">bbbb</div><brd-mark id="marker-12345"></brd-mark>'))

In [95]:
HTML(to_xml(Brd_Mark(id=new_id())))

# Colophon
----


In [1]:
import fastcore.all as FC
import nbdev
from nbdev.clean import nbdev_clean

In [2]:
if FC.IN_NOTEBOOK:
    nb_path = '14_bridge_helpers.ipynb'
    # nbdev_clean(nb_path)
    nbdev.nbdev_export(nb_path)
